In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("DateFruit_Dataset.csv")

In [3]:
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


In [4]:
df.isnull().sum()

AREA             0
PERIMETER        0
MAJOR_AXIS       0
MINOR_AXIS       0
ECCENTRICITY     0
EQDIASQ          0
SOLIDITY         0
CONVEX_AREA      0
EXTENT           0
ASPECT_RATIO     0
ROUNDNESS        0
COMPACTNESS      0
SHAPEFACTOR_1    0
SHAPEFACTOR_2    0
SHAPEFACTOR_3    0
SHAPEFACTOR_4    0
MeanRR           0
MeanRG           0
MeanRB           0
StdDevRR         0
StdDevRG         0
StdDevRB         0
SkewRR           0
SkewRG           0
SkewRB           0
KurtosisRR       0
KurtosisRG       0
KurtosisRB       0
EntropyRR        0
EntropyRG        0
EntropyRB        0
ALLdaub4RR       0
ALLdaub4RG       0
ALLdaub4RB       0
Class            0
dtype: int64

In [5]:
df.shape

(898, 35)

In [6]:
X = df.drop("Class", axis=1)
y = df["Class"]

In [7]:
df["Class"].unique()

<StringArray>
['BERHI', 'DEGLET', 'DOKOL', 'IRAQI', 'ROTANA', 'SAFAVI', 'SOGAY']
Length: 7, dtype: str

In [8]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [9]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### ANN

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [12]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [13]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [14]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [15]:
# Build our Model

class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(X.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 7),
        )
    
    def forward(self, x):
        return self.model(x)

In [16]:
model = ANN()

# loss & optim
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [17]:
# Training the NN

epochs = 100
for epoch in range(epochs):
    model.train()

    running_loss = 0.0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criteria(outputs, yb)
        loss.backward()
        optimizer.step() # params update
        running_loss = loss.item()

    train_loss = running_loss / len(train_loader)

    print(f"Epoch = {epoch+1} / {epochs}, loss = {train_loss}")

Epoch = 0 / 100, loss = 0.05145464254462201
Epoch = 1 / 100, loss = 0.03437131124994029
Epoch = 2 / 100, loss = 0.01455083878143974
Epoch = 3 / 100, loss = 0.014744641988173775
Epoch = 4 / 100, loss = 0.02462544648543648
Epoch = 5 / 100, loss = 0.01714246039805205
Epoch = 6 / 100, loss = 0.00857609899147697
Epoch = 7 / 100, loss = 0.015313986850821453
Epoch = 8 / 100, loss = 0.005518834875977557
Epoch = 9 / 100, loss = 0.010012591662614242
Epoch = 10 / 100, loss = 0.00473511542962945
Epoch = 11 / 100, loss = 0.015309927256210991
Epoch = 12 / 100, loss = 0.005096617276253907
Epoch = 13 / 100, loss = 0.01871527666630952
Epoch = 14 / 100, loss = 0.004724280341811802
Epoch = 15 / 100, loss = 0.01955873940301978
Epoch = 16 / 100, loss = 0.009943908971288929
Epoch = 17 / 100, loss = 0.004129061556380728
Epoch = 18 / 100, loss = 0.015414406423983366
Epoch = 19 / 100, loss = 0.004787536095017972
Epoch = 20 / 100, loss = 0.0031375463889992757
Epoch = 21 / 100, loss = 0.0037405620450558868
Epoch

In [18]:
# Evaluate 
model.eval()

total = 0
correct = 0

with torch.no_grad():
    for xb, yb in test_loader:
        outputs = model(xb) #[0.2, 0.5, 1.3, -0.5, ...] -> 7 values
        max_val, predicted = torch.max(outputs, 1) 

        correct += (predicted == yb).sum().item()
        total += yb.size(0) # acutal samples in each batch

print("Total Values: ", total)
print("Correct Values: ", correct)

Total Values:  180
Correct Values:  174


In [20]:
print("Accuracy: ", (correct / total) * 100)

Accuracy:  96.66666666666667
